<a href="https://colab.research.google.com/github/yexf308/AppliedStochasticProcess/blob/main/HW/HW3/560Sp23HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import numpy.linalg as LA
import codecs, json
from time import time
import plotly.express as px

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import re
import string
import requests
import collections
from collections import Counter

$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$

# Homework 3
## Homework guideline

- The deadline is March 31st 10:30am. Submission after the deadline will not be graded. 

- Submit your work(your reasoning and your code) as a SINGLE .ipynb document. Please rename the document as _HW1_YOURNAME.ipynb_ (for example, _HW1_FELIX.ipynb_). You are responsible for checking that you have correctly submitted the correct document. If your code cannot run, you may receive NO point. 

- Please justify all short answers with a brief explanation. If you use latex command in the markdown, **2 points** bonus will be awarded.   

- You only use the Python packages included in the following cell. You are not allowed to use other advanced package or modules unless you are permitted to. 

- In your final submission include the plots produced by the unedited code as presented below, as well as any additional plots produced after editing the code during the course of a problem. You may find it necessary to copy/paste relevant code into additional cells to accomplish this.

- Feel free to use the lecture notes and other resources. But you
must understand, write, and hand in your own answers. In addition, you must write and submit
your own code in the programming part of the assignment (we may run your code).
If you copy someone else homework solution, both of you may receive ZERO point. 


- Colab is preferred. However, if you use Anaconda, please download the .mat file locally and save it to the same folder as this homework file. 

# Q1: Theory (40pt)



## Q1.1 Coin Triplets I (20pt)
**If you cannot solve Q1.1, you may consider to use Monte Carlo Simulation to get numerical results. If your simulation is correct, you can still get most of points.** 

If you keep on tossing a fair coin, 
- What is the expected number of tosses such that you can have HHH (heads heads heads) in a row? 

- What is the expected number of tosses to have THH (tails heads heads) in a row?


# Your Solution:



---


## Q1.2 (10pt)
Suppose two distinct states $i, j$ of a Markov chain satisfy
$$ P(T_j< T_i|X_0=i)= P(T_i < T_j | X_0=j)$$
where $T_j=\inf\{n\ge 1: X_n=j\}$, which is the first return time to state $j$. Show that, if $X_0 = i$ the expected number of visits to $j$ prior to re-visiting $i$ is one, i.e.,
$$\mb{E}_i[\sum_{n=1}^\infty \m{I}(X_n=j) \m{I}(n\le T_i)]=1 $$


# Your Solution:



---


## Q1.3: Reversiblility (10pt)

A Markov chain on $S=\{0,1,\dots, N\}$ has transition probabilities $P(0,0) = 1-\lambda_0, P(i,i+1)=\lambda_i, P(i+1, i) =\mu_{i+1}$ for $i=0,\dots, N-1$ and $P(N,N)=1-\mu_N$. Show that the process
is reversible in equilibrium.  

# Your Solution:




---



---

# Q2: Beam Search (20pt)
**This problem can be used to present in showcase day as extra credits.**

In class, we didn't implement the last step in Beam Search so it will output the best sequence until it hit the stop_token or reaches the maximum step_times. Note the last step is 
- In the end, we obtain the set of final candidate output sequences based on these sequences. we choose the sequence with the highest of the following score as the output sequence:
$$\frac{1}{L^\alpha} \log p(X_1, X_2, \dots, X_L | X_0)= \frac{1}{L^\alpha}\sum_{t=1}^L \log p(X_{t}|X_{t-1}) $$
where $L$ is the length of the final candidate sequence and $\alpha$ is usually set to 0.75. Since a longer sequence has more logarithmic terms in the summation, the term $L^\alpha$ in the denominator penalizes long sequences.

In this question, you are going to modify ```generate_status``` function in the ```SimpleMarkovModel``` class. Then we are going to test with the same dataset in class. 



In [ ]:
class SimpleMarkovModel(object):
    def __init__(self, status_num=None):
        self.pi = np.zeros(shape=(status_num))
        self.P  = np.zeros(shape=(status_num, status_num))

    def fit(self, x):
        if type(x[0]) == list:
            for clist in x:
                self.pi[clist[0]] += 1
                for cindex in range(0, len(clist) - 1):
                    self.P[clist[cindex ], clist[cindex + 1]] += 1
        else:
            for index in range(0, len(x) - 1):
                self.pi[x[index]] += 1
                self.P[x[index ], x[index + 1]] += 1
        # normalization
        self.pi = self.pi / np.sum(self.pi)
        normalization = np.sum(self.P, axis=1) 
        normalization[normalization == 0] = 1  
        self.P = self.P / normalization[:, np.newaxis]  


    def generate_status(self, step_times=10, stop_status=None, set_start_status=None, beam_num=5, alpha = 0.75):
        """
        full beam search
        :param step_times: maximum step_times
        :param stop_status: list of stoping state
        :param set_start_status: set the initial state manually 
        :param beam_num: only when search_type="beam" to keep top k
        :return:
        """
        ### TO DO:Modify the following code to choose the sequences of step_times*beam_num with the score given above!

        if stop_status is None:
            stop_status = []
  
        start_status = np.random.choice(len(self.pi.reshape(-1)),
                                        p=self.pi.reshape(-1)) if set_start_status is None else set_start_status
    
        rst = [start_status]
        top_k_rst = [[start_status]]
        top_k_prob = [0.0]
        for _ in range(0, step_times):
            new_top_k_rst = []
            new_top_k_prob = []
            for k_index, k_rst in enumerate(top_k_rst):
                  k_rst_last_status = k_rst[-1]
                  # get top k largest idx 
                  top_k_idx = self.P[k_rst_last_status, : ].argsort()[::-1][0:beam_num]
                  for top_k_status in top_k_idx:
                    new_top_k_rst.append(k_rst + [top_k_status])
                    new_top_k_prob.append(top_k_prob[k_index] + np.log(1e-12+self.P[k_rst_last_status, top_k_status]))

            # sort all beam_num*beam_num results and get the top beam_num 
            top_rst_idx = np.asarray(new_top_k_prob).argsort()[::-1][0:beam_num]
            rst = new_top_k_rst[top_rst_idx[0]]
            # update
            top_k_rst = []
            top_k_prob = []
            for top_idx in top_rst_idx[:beam_num]:
              if new_top_k_rst[top_idx][-1] in stop_status:
                  rst = new_top_k_rst[top_idx]
                  break
              else:
                  top_k_rst.append(new_top_k_rst[top_idx])
                  top_k_prob.append(new_top_k_prob[top_idx])  

        return rst


In [ ]:
url = "https://raw.githubusercontent.com/probml/probml-data/main/data/timemachine.txt"
response = requests.get(url)
data = response.text
lines = [s + "\n" for s in response.text.split("\n")]
raw_dataset = [re.sub("[^A-Za-z]+", " ", st).lower().split() for st in lines]

# Concat sentences into single string of chars
# skip blank lines
sentences = [s for s in raw_dataset if s]

# Add the start and stop tokens to each sentence in the file
START_TOKEN = '<s>'
STOP_TOKEN = '</s>'
sentence_list = []
for sentence in sentences:
  sentence_list.append([START_TOKEN] + sentence + [STOP_TOKEN])

# Building the dict and the map from the words to integers for the purpose of training.
word2idx={}
idx2word={}
idx=0
for line in sentence_list:
    for word in line:
        if word not in word2idx:
            word2idx[word]=idx
            idx2word[idx]=word
            idx+=1

# Transform the words in each sentence to integers
train_data=[]
for line in sentence_list:
    train_data.append([word2idx[word] for word in line])

smm=SimpleMarkovModel(status_num=len(word2idx))
smm.fit(train_data)



In [ ]:
idx_list=smm.generate_status(set_start_status=word2idx['you'] , stop_status=[word2idx[word] for word in ['</s>']])
print([idx2word[idx] for idx in idx_list])

['you', 'cannot', 'move', 'freely', 'in', 'the', 'time', 'traveller', 's', 'pause', 'that']




---



---


# Q3: Natural Language Processing (40pt)
**This problem can be used to present in showcase day as extra credits.**

One of applications of Markov chain in natural language processing is language model. In particular, we will work on uni-gram and bi-gram. 

The following data set provides you with the training data files (a subset of the One Billion Word Language Modeling Bench-
mark). Each line in each file contains a whitespace-tokenized sentence.

- 1b benchmark.train.tokens: data for training your language models.

I have precessed these dataset for the purposed of this problem. The total number of unique words is 80661. In bi-gram model, the size of the transition matrix is $80661\times 80661$. It is impossible the store these matrices directly and we have to take advantage of the sparsity of the transition matrix. 

In [ ]:
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true -O 1b_benchmark.train.tokens
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.test.tokens?raw=true -O 1b_benchmark.test.tokens


--2023-03-30 01:28:25--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8422714 (8.0M) [text/plain]
Saving to: ‘1b_benchmark.train.tokens’

1b_benchmark.train. 100%[===================>]   8.03M  41.3MB/s    in 0.2s    

2023-03-30 01:28:26 (41.3 MB/s) - ‘1b_benchmark.train.tokens’ saved [8422714/8422714]

--2023-03-30 01:28:26--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.test.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [ ]:
START_TOKEN = "<s>"
STOP_TOKEN = "</s>"

class FileParser:
    def __init__(self, train_file="1b_benchmark.train.tokens", test_file="1b_benchmark.test.tokens"):
        self.TRAIN_FILE = train_file
        self.TEST_FILE  = test_file
    
    def get_train_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE))

    def get_test_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE))    
    
    def get_train_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE), flatten=False)  

    def get_test_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE), flatten=False)      
    
    def _flatten(self, l):
        return [word for sublist in l for word in sublist]

    def _tokenize(self, sentence_list, flatten=True):
        tokenized_sentences = [re.split("\s+", sentence.strip()) for sentence in sentence_list]

        if flatten:
            return self._flatten(tokenized_sentences)
        
        return tokenized_sentences

    def _get_sentences(self, file_path):
        
        l = []
        with open(file_path, "r") as f:
            l = f.readlines()
        
        # Add the start and stop tokens to each sentence in the file
        sentence_list = []
        for sentence in l:
            sentence_list.append(START_TOKEN + " " + sentence + " " + STOP_TOKEN)
        
        return sentence_list

In [ ]:
fp = FileParser()
train_tokens = fp.get_train_file_sentence_tokens()
test_tokens = fp.get_test_file_sentence_tokens()
# The fourth line of training set
print(train_tokens[3])

['<s>', 'Abu', 'Dhabi', 'is', 'going', 'ahead', 'to', 'build', 'solar', 'city', 'and', 'no', 'pollution', 'city', '.', '</s>']


In [ ]:
# Building the dict and the map from the words to integers for the purpose of training.
word2idx={}
idx2word={}
idx=0
for line in train_tokens:
    for word in line:
        if word not in word2idx:
            word2idx[word]=idx
            idx2word[idx]=word
            idx+=1

# Transform the words in each sentence to integers
train_data=[]
for line in train_tokens:
    train_data.append([word2idx[word] for word in line])

# there are 80661 unique words. 
len(word2idx)

80661



---



---


**Your task:** Complete the code in the ```LanguageModel``` class as follows. 

1. (10pt) Implement **Laplace Smoothing** with the smoothing factor =1 in the function ```get_unigram_probability``` and ```get_bigram_probability```. In the bigram model, try to implement it as efficient as possible. **Hint:** For bigrams $(t_1,t_2)$ which do not occur in the sample, what is $p(t_2|t_1)$? For fixed $t_1$, are these probability the same?

2. (10pt) Complete functions ```get_unigram_sentence_log_probability``` and ```get_bigram_sentence_log_probability``` to calculate the log probability of the given sentence. You need to consider the situation that when Laplace Smoothing is true and the situation that when Laplace Smoothing is false. 


3. (10pt) To make your language model work better, you will implement linear interpolation smoothing between
unigram, bigram. 
\begin{align}
p'(t_2|t_1) = \lambda_1 p(t_2) + \lambda_2 p(t_2|t_1)
\end{align}
where $p'$ represents the smoothed probability, the hyperparameters $\lambda_1, \lambda_2$ are weights on the unigram,
bigram language models, respectively. So $\lambda_1+\lambda_2= 1$. 
Complete functions ```get_linear_interpolation_probability``` and ```get_linear_interpolation_sentence_log_probability``` with $\mm\lambda = (\lambda_1, \lambda_2)$ stored in ```linear_interpolation_factors```. 

4. (10pt) Testing the smoothed probability with several setences in the testing dataset. You should test when laplace smoothing is True and when laplace smoothing is False. You might find some words in testing dataset are not appeared in training set and you can set a default probability for this situation.  

In [ ]:
class LanguageModel(object):
    def __init__(self, laplace_smoothing=False, laplace_smoothing_factor=1,linear_interpolation_factors=(0.3, 0.7)):
        self.unigram_freqs = {}
        self.unigram_corpus_length = 0
        self.num_unique_unigrams = 0

        self.bigram_freqs = {}
        self.bigram_corpus_length = 0
        self.num_unique_bigrams = 0

        self.laplace_smoothing = laplace_smoothing
        self.laplace_smoothing_factor = laplace_smoothing_factor
        
    def fit_unigram(self, tokens): 
        for clist in tokens: 
          for cindex in range(len(clist)):
            t = clist[cindex]
            self.unigram_freqs[(t)] = self.unigram_freqs.get((t),0) + 1

        self.unigram_corpus_length = sum(list(self.unigram_freqs.values()))
        self.num_unique_unigrams = len(list(self.unigram_freqs.keys()))

    def fit_bigram(self, tokens):
        
      for clist in tokens:
        for cindex in range(0, len(clist) - 1):
          t1, t2 =  clist[cindex], clist[cindex+1]
          self.bigram_freqs[(t1, t2)] = self.bigram_freqs.get((t1, t2), 0) + 1

      self.bigram_corpus_length = sum(list(self.bigram_freqs.values())) 
      self.num_unique_bigrams = len(list(self.bigram_freqs.keys()))   

    def get_unigram_probability(self, unigram): 

      if self.laplace_smoothing:
        ### TO DO: ###
        return
      else:
        prob_numerator   = self.unigram_freqs.get(unigram, 0) 
        prob_denominator = self.unigram_corpus_length
        prob = float(prob_numerator) / float(prob_denominator)  

        return prob

    def get_bigram_probability(self, bigram):
      t1, t2 = bigram 

      if self.laplace_smoothing:
        ### TO DO: ###
        return
      else:  
        prob_numerator   = self.bigram_freqs.get((t1, t2), 0)
        prob_denominator = self.unigram_freqs.get(t1, 0) 
        if prob_denominator == 0: 
            print(f"Error: 'get_bigram_probability()' has a denominator of 0 for {bigram}")
            return float('inf') 
        
        prob = float(prob_numerator) / float(prob_denominator)
        return prob 

    def get_linear_interpolation_probability(self, bigram):
      ### TO DO: ###
      return    

    def get_unigram_sentence_log_probability(self, sentence):
      ### TO DO: ###
      return 
    
    def get_bigram_sentence_log_probability(self, sentence):
      ### TO DO: ###
      return 

    def get_linear_interpolation_sentence_log_probability(self, sentence):  
      ### TO DO: ###
      return




In [ ]:
# Your code starts here. 
LM =LanguageModel()
LM.fit_bigram(train_data)
LM.fit_unigram(train_data)
LM.bigram_corpus_length

1622905

In [ ]:
LM.bigram_freqs.get((0, 1), 0)

43

In [ ]:
LM.unigram_freqs.get(0, 0) 

61530

In [ ]:
LM.get_bigram_probability((0,1))

0.0006988460913375589